## Нужно подготовить датасет для обучения: 
У нас имеются данные за каждые 6 часов;

Каждые 4 тензора соединяем в один, вот наша обучающая выборка для сетки;

In [6]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import glob

paths = glob.glob("data/Texas_gribs/land/*.grib")
DSs = []
for path in paths:
    DSs.append(xr.open_dataset(path, engine="cfgrib"))
#ds = xr.open_dataset("data/Texas_gribs/land/adaptor.mars.internal-1657472028.8672202-26530-12-3c74ef5c-ec79-4ac5-999d-b0616bc9dc51.grib", engine="cfgrib")

skipping variable: paramId==205 shortName='ro'
Traceback (most recent call last):
  File "E:\anaconda\envs\hail_project\lib\site-packages\cfgrib\dataset.py", line 660, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "E:\anaconda\envs\hail_project\lib\site-packages\cfgrib\dataset.py", line 591, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='time' value=Variable(dimensions=('time',), data=array([1577836800, 1577858400, 1577880000, ..., 1640930400, 1640952000,
       1640973600])) new_value=Variable(dimensions=('time',), data=array([1577815200, 1577858400, 1577901600, ..., 1640844000, 1640887200,
       1640930400]))
skipping variable: paramId==205 shortName='ro'
Traceback (most recent call last):
  File "E:\anaconda\envs\hail_project\lib\site-packages\cfgrib\dataset.py", line 660, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "E:\anaconda\envs\hail_project\lib\si

In [24]:
def prepare_full_train_data(aerology_path: str, land_path: str, one_day: bool = False):
    aerology_paths = glob.glob(aerology_path + "/*.grib")
    land_paths = glob.glob(land_path + "/*.grib")
    full_train_days = []
    for a_path, l_path in zip(land_paths, aerology_paths):
        full_train_days.append(np.concatenate([prepare_train_data(l_path, one_day),
                                               prepare_train_data(a_path, one_day)]))
    full_train_days = np.concatenate(full_train_days, axis=0)
    return full_train_days

def prepare_train_data(path: str, one_day: True):

    ds = xr.open_dataset(path, engine="cfgrib")
    vars = list(ds.data_vars)
    nps = {}.fromkeys(vars)

    for var in vars:
        nps[var] = ds[var].to_numpy()

    train_day = []
    time = ds.dims["time"]
    day_var = {}.fromkeys(vars)
    del ds

    for day in range(time * 6 // 24):
        for var in vars:
            day_var[var] = []
        for hour_period in range(4):
            for var in vars:
                day_var[var].append(nps[var][day * 4 + hour_period])
        train_day.append(np.array([day_var[var] for var in vars]).reshape(-1, 41, 65))
        if one_day:
            break

    train_day = np.array(train_day)
    return train_day

train_day = prepare_full_train_data("data/Texas_gribs/land", one_day=True)
train_day.shape

TypeError: prepare_full_train_data() missing 1 required positional argument: 'land_path'

In [20]:
DSs[0]["d2m"].to_numpy()

array([[[263.01218, 262.4106 , 261.8149 , ..., 266.52975, 266.54147,
         266.4067 ],
        [262.43796, 263.45944, 262.12936, ..., 266.9946 , 267.21335,
         266.94382],
        [262.186  , 262.46335, 262.35397, ..., 266.52   , 266.3403 ,
         266.45554],
        ...,
        [283.9907 , 282.6528 , 281.2817 , ..., 282.82663, 282.72116,
         282.66257],
        [285.22116, 283.30515, 281.93405, ..., 283.78952, 283.76022,
         283.7153 ],
        [286.13132, 284.2114 , 283.32858, ..., 284.57663, 284.57272,
         284.55905]],

       [[261.04974, 255.55757, 252.7646 , ..., 267.48138, 267.6474 ,
         267.6103 ],
        [258.2177 , 259.8974 , 257.39935, ..., 268.2021 , 268.52435,
         268.2099 ],
        [256.21967, 258.78412, 258.6103 , ..., 267.88763, 267.74115,
         267.7685 ],
        ...,
        [281.01654, 279.8017 , 278.87592, ..., 282.8056 , 282.59076,
         282.38763],
        [281.997  , 280.54193, 279.6103 , ..., 284.39935, 284.2685 ,
   

In [8]:
DSs[1]

<xarray.Dataset>
Dimensions:     (time: 2920, latitude: 41, longitude: 65)
Coordinates:
    number      int32 ...
  * time        (time) datetime64[ns] 2018-01-01 ... 2019-12-31T18:00:00
    step        timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 37.0 36.75 36.5 36.25 ... 27.5 27.25 27.0
  * longitude   (longitude) float64 -109.0 -108.8 -108.5 ... -93.5 -93.25 -93.0
    valid_time  (time) datetime64[ns] ...
Data variables:
    d2m         (time, latitude, longitude) float32 ...
    cape        (time, latitude, longitude) float32 ...
    sp          (time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2022-07-12T21:08 GRIB to CDM+CF via cfgrib-0.9.1...

In [9]:
DSs[2]

<xarray.Dataset>
Dimensions:     (time: 2924, latitude: 41, longitude: 65)
Coordinates:
    number      int32 ...
  * time        (time) datetime64[ns] 2016-01-01 ... 2017-12-31T18:00:00
    step        timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 37.0 36.75 36.5 36.25 ... 27.5 27.25 27.0
  * longitude   (longitude) float64 -109.0 -108.8 -108.5 ... -93.5 -93.25 -93.0
    valid_time  (time) datetime64[ns] ...
Data variables:
    d2m         (time, latitude, longitude) float32 ...
    cape        (time, latitude, longitude) float32 ...
    sp          (time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2022-07-12T21:08 GRIB to CDM+CF via cfgrib-0.9.1...

In [2]:
ds

<xarray.Dataset>
Dimensions:     (time: 2924, latitude: 41, longitude: 65)
Coordinates:
    number      int32 ...
  * time        (time) datetime64[ns] 2016-01-01 ... 2017-12-31T18:00:00
    step        timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 37.0 36.75 36.5 36.25 ... 27.5 27.25 27.0
  * longitude   (longitude) float64 -109.0 -108.8 -108.5 ... -93.5 -93.25 -93.0
    valid_time  (time) datetime64[ns] ...
Data variables:
    d2m         (time, latitude, longitude) float32 ...
    cape        (time, latitude, longitude) float32 ...
    sp          (time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2022-07-12T21:02 GRIB to CDM+CF via cfgrib-0.9.1...

In [18]:
#######

# Дописать вот эту функцию, для обработки сырых данных

#######
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

path = "Texas_gribs/cutted_aerology/adaptor.mars.internal-1657555490.0866394-25715-6-f2c5a5d2-4ce8-4a48-ad21-5bfc352828a1.grib"
def prepare_train_data(path):
    
    ds = xr.open_dataset(path, engine="cfgrib")
    np_z = ds.z.to_numpy()
    np_q = ds.q.to_numpy()
    np_t = ds.t.to_numpy()
    np_u = ds.u.to_numpy()
    np_v = ds.v.to_numpy()
    time = ds.dims["time"]
    del ds
    train_day = []
    for day in range(time * 6 // 24 ):
        day_z = []
        day_q = []
        day_t = []
        day_u = []
        day_v = []
        for hour_period in range(4):
            day_z.append(np_z[day*4 + hour_period])
            day_q.append(np_q[day*4 + hour_period])
            day_t.append(np_t[day*4 + hour_period])
            day_u.append(np_u[day*4 + hour_period])
            day_v.append(np_v[day*4 + hour_period])
        train_day.append(np.array([day_z, day_q, day_t, day_u, day_v]).reshape(-1, 41, 65))
        break
    train_day = np.array(train_day)
    return train_day

#train_day, train_cape, train_sp, train_d2m = prepare_train_data(path)
train_day = prepare_train_data(path)

In [19]:
train_day.shape

(1, 60, 41, 65)

In [9]:
def prepare_train_data_land(path):

    ds = xr.open_dataset(path, engine="cfgrib")
    np_d2m = ds.d2m.to_numpy()
    np_cape = ds.cape.to_numpy()
    np_sp = ds.sp.to_numpy()
    train_day = []
    time = ds.dims["time"]
    del ds
    for day in range(time * 6 // 24):
        day_d2m = []
        day_cape = []
        day_sp = []

        for hour_period in range(4):
            day_d2m.append(np_d2m[day*4 + hour_period])
            day_cape.append(np_cape[day*4 + hour_period])
            day_sp.append(np_sp[day*4 + hour_period])
        train_day.append(np.array([day_sp, day_cape, day_d2m]).reshape(-1, 41, 65))
        break
    train_day = np.array(train_day)
    return train_day

path = "Texas_gribs/land/adaptor_mars_internal_1657472008_0885537_3991_5_4cbdaa1c_21f3_4aed.grib"

tr_day = prepare_train_data_land(path)

skipping variable: paramId==205 shortName='ro'
Traceback (most recent call last):
  File "E:\anaconda\envs\hail_project\lib\site-packages\cfgrib\dataset.py", line 660, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "E:\anaconda\envs\hail_project\lib\site-packages\cfgrib\dataset.py", line 591, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='time' value=Variable(dimensions=('time',), data=array([1514764800, 1514786400, 1514808000, ..., 1577772000, 1577793600,
       1577815200])) new_value=Variable(dimensions=('time',), data=array([1514743200, 1514786400, 1514829600, ..., 1577685600, 1577728800,
       1577772000]))


In [13]:
tr_day.shape

(1, 12, 41, 65)

In [20]:
np.concatenate([tr_day, train_day], axis=1).shape 

(1, 72, 41, 65)

In [29]:
leap_years = [2020, 2016]
years = np.unique(ds.time.dt.year.to_numpy())
np_d2m = ds.d2m.to_numpy()
np_cape = ds.cape.to_numpy()
np_sp = ds.sp.to_numpy()
train_d2m = []
train_cape = []
train_sp = []
for day in range(ds.dims["time"] * 6 / 24 ):
    day_d2m = []
    day_cape = []
    day_sp = []
    for hour_period in range(4):
        day_d2m.append(np_d2m[day*4 + hour_period])
        day_cape.append(np_cape[day*4 + hour_period])
        day_sp.append(np_sp[day*4 + hour_period])
    train_d2m.append(np.array(day_d2m))
    train_cape.append(np.array(day_cape))
    train_sp.append(np.array(day_sp))
train_d2m = np.array(train_d2m)
train_cape = np.array(train_cape)
train_sp = np.array(train_sp)

In [32]:
np.array(train_d2m).shape

(4, 41, 65)

In [40]:
ds.dims["time"]

2924

In [20]:
np.unique(ds.time.dt.year.to_numpy())

array([2020, 2021], dtype=int64)